In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv("filtered_usable_df.csv", sep=";")
print(df.shape)

(5742878, 16)


In [5]:
df.head()

,Title,Author,Category,Genre,Language,Status,Published,Updated,Rating,Summary,word_count,chapter_count,story_id,processed_title,processed_summary,combined_text
0,The Distant Strains of Triumph,SpartanDelta2,"Halo, Mass Effect","Drama, Sci-Fi",English,Completed,2013-07-15,2013-07-15,M,"A project for my class, a challenge from my te...",1180,1,9493934,"['distant', 'strain', 'triumph']","['project', 'class', 'challenge', 'teacher', '...","['distant', 'strain', 'triumph', 'project', 'c..."
1,Mass Effect: Beyond,Cunningham-Hughes,"Halo, Mass Effect","Adventure, Sci-Fi",English,Completed,2012-10-01,2012-10-01,T,A hardcore Mass Effect fan's retelling of the ...,9971,1,8573953,"['mass', 'effect']","['hardcore', 'mass', 'effect', 'fan', 'retelli...","['mass', 'effect', 'hardcore', 'mass', 'effect..."
2,Chaos Infinitium,Sysero of Cain,"Halo, Mass Effect","Adventure, Sci-Fi",English,Completed,2011-03-11,2011-11-17,T,First Contact never seems to go well. But thin...,19447,5,6816070,"['chaos', 'infinitium']","['contact', 'thing', 'worse', 'happens', 'civi...","['chaos', 'infinitium', 'contact', 'thing', 'w..."
3,Tools of Defiance,Magisking,"Halo, Mass Effect","Angst, Tragedy",English,Completed,2013-12-22,2013-12-22,T,A One-shot that takes place in the Defiance un...,997,1,9946669,"['tool', 'defiance']","['oneshot', 'take', 'place', 'defiance', 'univ...","['tool', 'defiance', 'oneshot', 'take', 'place..."
4,Mass Effect New Origins V2,erttheking,"Halo, Mass Effect","Romance, Sci-Fi",English,Completed,2010-10-05,2012-06-09,T,"On the dawn of the 27th century, the UNSC disc...",442687,109,6376514,"['mass', 'effect', 'new', 'origin', 'v']","['dawn', 'th', 'century', 'unsc', 'discovers',...","['mass', 'effect', 'new', 'origin', 'v', 'dawn..."


In [7]:
# Define features and target variable
features = ['combined_text', 'processed_title', 'processed_summary', 'Genre']
target = 'Category'

X = df[features]
y = df[target]

In [8]:
text_columns = ['combined_text', 'processed_title', 'processed_summary']

# Initialize a TfidfVectorizer for text processing
tfidf = TfidfVectorizer(max_features=5000)  # Adjust max_features based on dataset size


In [9]:
categorical_columns = ['Genre']

# Initialize a OneHotEncoder for categorical processing
one_hot = OneHotEncoder(handle_unknown='ignore')


In [10]:
from sklearn.compose import make_column_transformer

preprocessor = make_column_transformer(
    (tfidf, 'combined_text'),
    (tfidf, 'processed_title'),
    (tfidf, 'processed_summary'),
    (one_hot, ['Genre']),
    remainder='drop'  # Drop any unspecified columns
)


In [ ]:
classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [ ]:
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)


In [ ]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


In [ ]:
from joblib import dump

# Save the trained model or pipeline to a file
dump(pipeline, 'trained_pipeline.joblib')

print("Model saved successfully as 'trained_pipeline.joblib'.")